In [ ]:
import pandas as pd
from datetime import datetime, date
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from easydict import EasyDict
import joblib
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [ ]:
# Read in data
filename = "C:/Users/phili/Downloads/Seattle_Real_Time_Fire_911_Calls.csv"
raw_df = pd.read_csv(filename)

# Preprocessing

In [ ]:
def preprocessing(raw_df):
    """
    Prepocesses the data and returns dataframe
    :param raw_df: pd.Dataframe
    :return: df(pd.Dataframe)
    """
    # Divide date into Years,Months,Days,Hours
    raw_df["Datetime"]= raw_df["Datetime"].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
    raw_df["Year"] = raw_df["Datetime"].apply(lambda x: int(x.year))
    raw_df = raw_df[raw_df["Year"] >= 2014]
    raw_df["Month"] = raw_df["Datetime"].apply(lambda x: int(x.month))
    raw_df["Day"] = raw_df["Datetime"].apply(lambda x: int(x.day))
    raw_df["Hour"] =raw_df["Datetime"].apply(lambda x: int(x.hour))

    raw_df.dropna(inplace=True) # drop null values

    df = raw_df.copy()
    # drop features we won't use
    df.drop(["Address", "Type", "Report Location", "Incident Number"], axis=1, inplace=True)
    return df

# Feature Engineering

In [ ]:
def create_target_variable(df):
    """
    Creates the target variable: Hourly Call Volume (Number of Calls per hour)
    :param df: pd.Dataframe, containing the preprocessed data
    :return: df (with target variable)
    """
    y_m_d= df.apply(lambda x: (int(x["Year"]), int(x["Month"]),int(x["Day"]), int(x["Hour"])), axis=1)
    hourly_call_volume = df.groupby(by=["Year", "Month", "Day", "Hour"]).count()["Latitude"].to_dict()
    df["Hourly Call Volume"] = y_m_d.map(hourly_call_volume)
    return df

def add_daily_call_volume_feat(df):
    """
    Adds a new feature: Daily Call Volume (Number of calls per day)
    :param df: pd.Dataframe, containing the preprocessed data
    :return: df (with new feature)
    """
    m_d = df.apply(lambda x: (int(x["Year"]), int(x["Month"]),int(x["Day"])), axis=1)
    avg_daily_call_volumes = df.groupby(by=["Year", "Month", "Day"])["Hour"].count().to_dict() #.apply(lambda x: x/df["Year"].unique().size)
    df["Daily Call Volume"] = m_d.map(avg_daily_call_volumes)
    return df

def get_season(in_datetime):
    """
    Returns season of datetime object
    :param in_datetime: datetime.datetime
    :return: season (str): winter, spring, summer or fall
    """
    Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
    seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
               ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
               ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
               ('fall', (date(Y,  9, 23),  date(Y, 12, 20))),
               ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

    assert isinstance(in_datetime, datetime), "Not a datetime object!"
    in_datetime = in_datetime.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= in_datetime <= end)

def create_features(df):
    """
    Creates several new features and returns an Easydict containing the train and test data
    :param df: pd.Dataframe, containing preprocessed data
    :return: EasyDict containing both train and test data
    """
    df["Season"]= df["Datetime"].apply(lambda x: get_season(x))
    df["Weekday"]= df["Datetime"].apply(lambda x: datetime.weekday(x))
    df = add_daily_call_volume_feat(df)

    df = create_target_variable(df)

    df.sort_values(by='Datetime', inplace=True) # Sort by Date
    df.drop(["Datetime"], axis=1, inplace=True) # All necessary information already exported to other columns

    # We want to predict the hourly call volume of 2019
    df_train = df[df["Year"] < 2019]
    df_test = df[df["Year"] == 2019]

    target_train = df_train.pop("Hourly Call Volume")
    target_test = df_test.pop("Hourly Call Volume")

    return EasyDict(X_train=df_train,y_train=target_train, X_test=df_test, y_test=target_test)

# Training

In [ ]:
def training(x_train, y_train):
    """
    Trains the Decision Tree
    :param x_train: pd.Dataframe, containing the features of the training data
    :param y_train: pd.Series, containing the target variable of the training data
    """

    numerical_cols = x_train.select_dtypes('number').columns
    categorical_cols = pd.Index(np.setdiff1d(x_train.columns, numerical_cols))
    rng = np.random.RandomState(1)
    tree = AdaBoostRegressor(
        DecisionTreeRegressor(max_depth=5), n_estimators=300, random_state=rng)

    numerical_pipe = Pipeline([
        ('scaler', StandardScaler())])

    categorical_pipe = Pipeline([(
        'encoder', OneHotEncoder(drop='first', handle_unknown='error'))])

    preprocessors = ColumnTransformer(transformers=[
        ('num', numerical_pipe, numerical_cols),
        ('cat', categorical_pipe, categorical_cols)
    ])

    pipe = Pipeline([('preprocessors', preprocessors), ('tree', tree)])

    i=100000
    pipe.fit(x_train.iloc[:i,:], y_train.iloc[:i])

    joblib.dump(pipe, "./dec_tree_pipe.joblib")

# Testing

In [ ]:
def test_tree(x_train, y_train, x_test, y_test):
    """
    Evaluates the
    :param x_train: pd.Dataframe, containing the features of the training data
    :param y_train: pd.Series, containing the target variable of the training data
    :param x_test: pd.Dataframe, containing the features of the test data
    :param y_test: pd.Series, containing the target variable of the test data
    :return:
    """

    pipe = joblib.load("./dec_tree_pipe.joblib")

    train_pred = pipe.predict(x_train)
    train_loss = mean_squared_error(y_train, train_pred)
    train_R2_score = pipe.score(x_train, y_train)
    print('Training MSE Loss: {} \n Test R2 score {}'.format(train_loss, train_R2_score))

    test_pred = pipe.predict(x_test)
    test_loss = mean_squared_error(y_test, test_pred)
    test_R2_score  = pipe.score(x_test, y_test)
    print('Test MSE Loss: {} \n Test R2 score {}'.format(test_loss, test_R2_score))
    print(type(test_pred))
    return test_pred

In [ ]:
df = preprocessing(raw_df)

In [ ]:
data = create_features(df)

In [ ]:
i=100000
# Choose only a subset of training data in case of limited computational resources
data.X_train, data.y_train = shuffle(data.X_train, data.y_train, random_state=0)
data.X_train, data.y_train = data.X_train.iloc[:i,:], data.y_train.iloc[:i]

training(data.X_train, data.y_train)

In [ ]:
predictions = test_tree(data.X_train, data.y_train, data.X_test, data.y_test)

# Visualization

In [ ]:
df = pd.concat([data.X_test, data.y_test], axis=1) # useful to get Hourly Call Volume according to hours and days
df.reset_index(inplace=True)
y_test_2 = data.y_test.reset_index(drop=True)
indices = []
# Get Hourly Call Volume for
for day in range(1,8):
    for hour in range(0,24):
        indices.append(df[(df["Month"]==3) & (df["Day"]==day) & (df["Hour"]==hour)]["Hourly Call Volume"].index[0])

plt.plot(np.arange(7*24), y_test_2[indices])
plt.plot(np.arange(7*24), predictions[indices])
plt.xlabel("Hour")
plt.ylabel("Call Volume")
plt.title("Hourly Call Volume 1st week of March'19")
plt.legend(["True", "Prediction"])
plt.savefig("Hourly Call Volume 1st week March'19.png")
plt.show()